<a href="https://colab.research.google.com/github/mohammad-jamalaldeen/12-feb-2023-whisper-and-datasets/blob/main/12_feb_2023_Whisper_and_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Settings:
1. Runtime: TPU.
2. Keep connecting by the JavaScript code.
3. Mounting checkpoints to Google Drive.

In [ ]:
# function ConnectButton(){
#     console.log("Connect pushed");
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
# }
# setInterval(ConnectButton,60000);

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


For Unix, update the package `ffmpeg` to version 4:

In [ ]:
# !add-apt-repository -y ppa:jonathonf/ffmpeg-4
# !apt update
# !apt install -y ffmpeg

We'll employ several popular Python packages to fine-tune Whisper model.
We'll use `datasets` to download and prepare our training data and
`transformers` to load and train specific Whisper model. We'll also use
the `soundfile` package to pre-process audio files, `evaluate` and `jiwer` to
assess the performance of the model. Finally, we can use `gradio` to build a demo of the fine-tuned model.

In [ ]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-z6qd27ef
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-z6qd27ef
  Resolved https://github.com/huggingface/transformers to commit c836f77266be9ace47bff472f63caf71c0d11333
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6464552 sha256=5bd5f2326c615fe06383ed061d33f8d943f5847f1e2408661faf436a620bc199
  Stored in directory: /tmp/pip-ephem-wheel-cache-l0j3347t/wheels/42/68/45/c63edff61c292f2dfd4df4ef6522dcbecc603e7af82813c1d7
Successfully built transformers
Looking in indexes: https://pypi.org/simp

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
BASE_MODEL = "openai/whisper-tiny"

## Load Dataset

Using 🤗 Datasets, downloading and preparing data is extremely simple.
We can download and prepare the Common Voice splits in just one line of code.

First, ensure you have accepted the terms of use on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). Once you have accepted the terms, you will have full access to the dataset and be able to download the data locally.

We'll combine the `train` and `validation`
splits to give approximately 8 hours of training data. We'll use the 4 hours
of `test` data as our held-out test set:

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="test", use_auth_token=True)

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 38481
    })
    test: Dataset({
        features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
        num_rows: 10440
    })
})


Most ASR datasets only provide input audio samples (`audio`) and the
corresponding transcribed text (`sentence`). Common Voice contains additional
metadata information, such as `accent` and `locale`, which we can disregard for ASR.
Keeping the notebook as general as possible, we only consider the input audio and
transcribed text for fine-tuning, discarding the additional metadata information:

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 38481
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10440
    })
})


In [ ]:
sample = common_voice['train'][4]
print(sample)

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/88b9814b92fde328c0884b8c80e1bef34be70b365af58b508b5058d520736eb5/cv-corpus-11.0-2022-09-21/ar/clips/common_voice_ar_19227176.mp3', 'array': array([ 0.0000000e+00, -4.8836606e-14, -6.5283892e-14, ...,
       -2.8335210e-04, -2.5399099e-04, -1.3521209e-04], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'لديها ابنة تعزف على البيانو.'}


In [ ]:
from IPython.display import Audio, display
display(Audio(sample['audio']['path'], autoplay=False))
print(sample['sentence'])

لديها ابنة تعزف على البيانو.


## Prepare Feature Extractor, Tokenizer and Data

The ASR pipeline can be de-composed into three stages:
1) A feature extractor which pre-processes the raw audio-inputs
2) The model which performs the sequence-to-sequence mapping
3) A tokenizer which post-processes the model outputs to text format

In 🤗 Transformers, the Whisper model has an associated feature extractor and tokenizer,
called [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)
and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)
respectively.

We'll go through details for setting-up the feature extractor and tokenizer one-by-one!

### Load WhisperFeatureExtractor

The Whisper feature extractor performs two operations:
1. Pads / truncates the audio inputs to 30s: any audio inputs shorter than 30s are padded to 30s with silence (zeros), and those longer that 30s are truncated to 30s
2. Converts the audio inputs to _log-Mel spectrogram_ input features, a visual representation of the audio and the form of the input expected by the Whisper model

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(BASE_MODEL)

### Load WhisperTokenizer

The Whisper model outputs a sequence of _token ids_. The tokenizer maps each of these token ids to their corresponding text string. For Arabic, we can load the pre-trained tokenizer and use it for fine-tuning without any further modifications. We simply have to
specify the target language and the task. These arguments inform the
tokenizer to prefix the language and task tokens to the start of encoded
label sequences:

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(BASE_MODEL, language="Arabic", task="transcribe")

### Create A WhisperProcessor

To simplify using the feature extractor and tokenizer, we can _wrap_
both into a single `WhisperProcessor` class. This processor object
inherits from the `WhisperFeatureExtractor` and `WhisperProcessor`,
and can be used on the audio inputs and model predictions as required.
In doing so, we only need to keep track of two objects during training:
the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(BASE_MODEL, language="Arabic", task="transcribe")

### Prepare Data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

In [ ]:
common_voice

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 38481
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 10440
    })
})

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/88b9814b92fde328c0884b8c80e1bef34be70b365af58b508b5058d520736eb5/cv-corpus-11.0-2022-09-21/ar/clips/common_voice_ar_19225971.mp3', 'array': array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        2.3667126e-06, -2.5793133e-06,  6.8762515e-06], dtype=float32), 'sampling_rate': 48000}, 'sentence': 'محصول الطماطم جودته عالية.'}


Since
our input audio is sampled at 48kHz, we need to _downsample_ it to
16kHz prior to passing it to the Whisper feature extractor, 16kHz being the sampling rate expected by the Whisper model.

We'll set the audio inputs to the correct sampling rate using dataset's
[`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column)
method. This operation does not change the audio in-place,
but rather signals to `datasets` to resample audio samples _on the fly_ the
first time that they are loaded:

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [ ]:
print(common_voice["train"][0])

{'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/88b9814b92fde328c0884b8c80e1bef34be70b365af58b508b5058d520736eb5/cv-corpus-11.0-2022-09-21/ar/clips/common_voice_ar_19225971.mp3', 'array': array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 3.9961551e-06,
       8.9953237e-06, 6.5385566e-06], dtype=float32), 'sampling_rate': 16000}, 'sentence': 'محصول الطماطم جودته عالية.'}


Now we can write a function to prepare our data ready for the model:
1. We load and resample the audio data by calling `batch["audio"]`. As explained above, 🤗 Datasets performs any necessary resampling operations on the fly.
2. We use the feature extractor to compute the log-Mel spectrogram input features from our 1-dimensional audio array.
3. We encode the transcriptions to label ids through the use of the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We can apply the data preparation function to all of our training examples using dataset's `.map` method. The argument `num_proc` specifies how many multiprocesses to use. If the `.map` method hangs with multiprocessing, set `num_proc=1` and process the dataset sequentially.

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

In [ ]:
common_voice

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
data = load_dataset("arabic_speech_corpus")

Generating train split:   0%|          | 0/1813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Dataset arabic_speech_corpus downloaded and prepared to /root/.cache/huggingface/datasets/arabic_speech_corpus/clean/2.1.0/6d3f67a1b44f6a73a952b1130167cd94c31d0375918b431a10b6f68ae9145a05. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['file', 'text', 'audio', 'phonetic', 'orthographic'],
        num_rows: 1813
    })
    test: Dataset({
        features: ['file', 'text', 'audio', 'phonetic', 'orthographic'],
        num_rows: 100
    })
})

In [ ]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import login

In [ ]:
login("")

In [ ]:
data.push_to_hub("arbml/", private = True)